In [1]:
# Importar libs
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

numpy : 1.21.5
pandas: 1.4.4
plotly: 5.9.0



In [3]:
# Carrega os dados
df_incident = pd.read_excel('TFL Bus Safety.xlsx')
df_incident.head()

,Year,Date Of Incident,Route,Operator,Group Name,Bus Garage,Borough,Injury Result Description,Incident Event Type,Victim Category,Victims Sex,Victims Age
0,2015,2015-01-01,1,London General,Go-Ahead,Garage Not Available,Southwark,Injuries treated on scene,Onboard Injuries,Passenger,Male,Child
1,2015,2015-01-01,4,Metroline,Metroline,Garage Not Available,Islington,Injuries treated on scene,Onboard Injuries,Passenger,Male,Unknown
2,2015,2015-01-01,5,East London,Stagecoach,Garage Not Available,Havering,Taken to Hospital – Reported Serious Injury or...,Onboard Injuries,Passenger,Male,Elderly
3,2015,2015-01-01,5,East London,Stagecoach,Garage Not Available,None London Borough,Taken to Hospital – Reported Serious Injury or...,Onboard Injuries,Passenger,Male,Elderly
4,2015,2015-01-01,6,Metroline,Metroline,Garage Not Available,Westminster,Reported Minor Injury - Treated at Hospital,Onboard Injuries,Pedestrian,Female,Elderly


In [4]:
# 1- Qual a quantidade de incidentes por gênero?

df_incident['Victims Sex'].value_counts().reset_index()

,index,Victims Sex
0,Female,11847
1,Male,7709
2,Unknown,3602


In [5]:
# 2- Qual faixa etária esteve mais envolvida nos incidentes?

df_incident['Victims Age'].value_counts().reset_index()

,index,Victims Age
0,Adult,10754
1,Unknown,7135
2,Elderly,2769
3,Child,2181
4,Youth,319


In [6]:
# 3- Qual o percentual de incidentes por tipo de evento (Incident EventType)?
incident_sum = df_incident['Incident Event Type'].value_counts()
incident_perc = df_incident['Incident Event Type'].value_counts(normalize=True) * 100
incident_acum = df_incident['Incident Event Type'].value_counts(normalize=True).cumsum() * 100

dic = {
    'Event Type': incident_sum.index,
    'Total': incident_sum.values,
    'Percentage': np.around(incident_perc.values, 2),
    'Cumulative Percentage': np.around(incident_acum.values, 2),
}

df_event_type = pd.DataFrame(dic)

df_event_type


,Event Type,Total,Percentage,Cumulative Percentage
0,Slip Trip Fall,6981,30.15,30.15
1,Onboard Injuries,6563,28.34,58.49
2,Personal Injury,4596,19.85,78.33
3,Collision Incident,4166,17.99,96.32
4,Assault,590,2.55,98.87
5,Activity Incident Event,114,0.49,99.36
6,Vandalism Hooliganism,73,0.32,99.68
7,Safety Critical Failure,66,0.28,99.96
8,Fire,6,0.03,99.99
9,Robbery,3,0.01,100.00


In [7]:
# 4- Como foi a evolução de incidentes por mês ao longo do tempo?

# Multas por dia
multas_mes = df_incident['Date Of Incident'].value_counts().sort_index().reset_index()

# Extraindo o mes
multas_mes['Mes'] = pd.to_datetime(multas_mes['Date Of Incident']).dt.month

# Renomeia as colunas
multas_mes.columns = ['Data', 'Quantidade', 'Mes']

multas_mes.head()

,Data,Quantidade,Mes
0,2015-01-01,399,1
1,2015-02-01,371,1
2,2015-03-01,460,1
3,2015-04-01,470,1
4,2015-05-01,472,1


In [8]:
# 4- Como foi a evolução de incidentes por mês ao longo do tempo?
# Grafico de barras
px.bar(
    multas_mes,
    x='Data',
    y='Quantidade',
    title='Incidentes'
)

In [9]:
# 5- Quando o incidente foi “Collision Incident” em qual mês houve o maior 
# número de incidentes envolvendo pessoas do sexo feminino?

# GroupBy
df_grouped = df_incident[['Incident Event Type', 'Date Of Incident', 'Victims Sex']]\
.groupby(['Incident Event Type']).value_counts().reset_index()

# Extrai Mes e Ano
df_grouped['Mes'] = pd.to_datetime(df_grouped['Date Of Incident']).dt.month
df_grouped['Ano'] = pd.to_datetime(df_grouped['Date Of Incident']).dt.year

# Filtra as informações solicitadas no problema 
df_grouped = df_grouped[(df_grouped['Incident Event Type'] == 'Collision Incident') \
                       & (df_grouped['Victims Sex'] == 'Female')]

df_grouped.head()

,Incident Event Type,Date Of Incident,Victims Sex,0,Mes,Ano
166,Collision Incident,2016-11-01,Female,63,11,2016
171,Collision Incident,2016-09-01,Female,56,9,2016
175,Collision Incident,2017-08-01,Female,52,8,2017
178,Collision Incident,2017-07-01,Female,49,7,2017
182,Collision Incident,2016-06-01,Female,47,6,2016


In [10]:
# 6- Qual foi a média de incidentes por mês envolvendo crianças (Child)?

df_child = df_incident[['Victims Age', 'Date Of Incident']].value_counts().sort_index().reset_index()

# Extraindo dia e mes
df_child['Mes'] = pd.to_datetime(df_child['Date Of Incident']).dt.month
df_child['Ano'] = pd.to_datetime(df_child['Date Of Incident']).dt.year

df_child = df_child[(df_child['Victims Age'] == 'Child')]

df_child.columns = ['Idade', 'Data', 'Quantidade', 'Mes', 'Ano']

# Media movel
df_child['Media'] = df_child['Quantidade'].rolling(2).mean()

df_child.head()

,Idade,Data,Quantidade,Mes,Ano,Media
45,Child,2015-01-01,31,1,2015,NaN
46,Child,2015-02-01,27,2,2015,29.0
47,Child,2015-03-01,33,3,2015,30.0
48,Child,2015-04-01,36,4,2015,34.5
49,Child,2015-05-01,44,5,2015,40.0


In [11]:
# 6- Qual foi a média de incidentes por mês envolvendo crianças (Child)?
# Gráfico com a média de incidentes por mês envolvendo crianças

grid = make_subplots(rows=1, cols=1)

grid.add_trace(
    go.Scatter(
        x=df_child['Data'],
        y=df_child['Quantidade'],
        mode='lines',
        name='Quantidade'
    ),
    row=1, col=1
)

grid.add_trace(
    go.Scatter(
        x=df_child['Data'],
        y=df_child['Media'],
        mode='lines',
        name='Média'
    ),
    row=1, col=1
)

grid.update_layout(
    title='Análise de incidentes',
    
    #Ajustes Plotly
    legend=dict(
        orientation='h',
    )
)

In [12]:
# 7- Considerando a descrição de incidente como “Injuries treated on scene”  (coluna
# Injury Result Description), qual o total de incidentes de pessoas do sexo masculino e sexo feminino?

# GroupBy
df_grouped = df_incident[['Injury Result Description', 'Victims Sex']]\
.groupby(['Injury Result Description']).value_counts().reset_index()

df_grouped = df_grouped[(df_grouped['Injury Result Description'] == 'Injuries treated on scene')]

df_grouped

,Injury Result Description,Victims Sex,0
2,Injuries treated on scene,Female,8816
3,Injuries treated on scene,Male,5632
4,Injuries treated on scene,Unknown,2888


In [17]:
# 8- No ano de 2017 em qual mês houve mais incidentes com idosos (Elderly)?

df_elderly = df_incident

# Extraindo mes e ano
df_elderly['Ano'] = pd.to_datetime(df_incident['Date Of Incident']).dt.year
df_elderly['Mes'] = pd.to_datetime(df_incident['Date Of Incident']).dt.month

df_elderly = df_elderly[['Victims Age', 'Date Of Incident', 'Ano', 'Mes']]\
.groupby(['Ano']).value_counts().sort_index().reset_index()

df_elderly = df_elderly.rename(columns={0: 'total'})

df_elderly = df_elderly[(df_elderly['Ano'] == 2017 ) & (df_elderly['Victims Age'] == 'Elderly')]\
.sort_values(by='total', ascending=False)

df_elderly.head()

,Ano,Victims Age,Date Of Incident,Mes,total
150,2017,Elderly,2017-07-01,7,81
152,2017,Elderly,2017-09-01,9,78
146,2017,Elderly,2017-03-01,3,77
147,2017,Elderly,2017-04-01,4,75
151,2017,Elderly,2017-08-01,8,70


In [21]:
# 9- Considerando o Operador qual a distribuição de incidentes ao longo do tempo?

df_operator = df_incident[['Operator', 'Date Of Incident']].value_counts().sort_index().reset_index()

df_operator = df_operator.rename(columns={0: 'total'})

df_operator

,Operator,Date Of Incident,total
0,Abellio London,2015-01-01,11
1,Abellio London,2015-02-01,6
2,Abellio London,2015-03-01,7
3,Abellio London,2015-04-01,9
4,Abellio London,2015-05-01,11
...,...,...,...
831,Tower Transit,2018-06-01,19
832,Tower Transit,2018-07-01,11
833,Tower Transit,2018-08-01,11
834,Tower Transit,2018-09-01,14


In [22]:
# 9- Considerando o Operador qual a distribuição de incidentes ao longo do tempo?
fig = px.line(df_operator, x='Date Of Incident', y='total', color='Operator')
fig.show()

In [20]:
# 10- Qual o tipo de incidente mais comum com ciclistas?

# GroupBy
df_grouped = df_incident[['Incident Event Type', 'Victim Category']]\
.groupby(['Victim Category']).value_counts().reset_index()

df_grouped = df_grouped[(df_grouped['Victim Category'] == 'Cyclist')]

df_grouped

,Victim Category,Incident Event Type,0
23,Cyclist,Collision Incident,256
24,Cyclist,Personal Injury,8
25,Cyclist,Slip Trip Fall,7
26,Cyclist,Onboard Injuries,4
